<span style="font-size:2em; color:blue">
    Tema 13: Programas interactivos
</span>  

----------

[José A. Alonso](https://www.cs.us.es/~jalonso)  
[Departamento de Ciencias de la Computación e I.A.](https://www.cs.us.es)  
[Universidad de Sevilla](http://www.us.es)  
Sevilla, 9 de agosto de 2019

> __Notas:__ 
+ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-13.ipynb).
+ Se desactiva el [corrector estilo de Haskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint) en Jupyter.

In [1]:
:opt no-lint

> __Nota sobre ejecución intractiva en Jupyter__: 

> Para simular un entorno de línea de comandos interactivo para programas se ha adoptado la [método de James Brock](https://mybinder.org/v2/gh/jamesdbrock/learn-you-a-haskell-notebook/master?urlpath=lab/tree/learn_you_a_haskell/09-input-and-output.ipynb) consistente en el uso de las funciones 
> + `withStdin` que toma una cadena y ejecuta una acción IO con esa cadena como entrada estándar. 
> + `catchPrint`que captura una excepción y la imprime en la salida estándar.

In [2]:
import System.IO
import GHC.IO.Handle
import Control.Exception
import System.Directory

withStdin :: String -> IO a -> IO a
withStdin s action = do
    writeFile "stdin.txt" s
    finally
        (bracket 
            (openFile "stdin.txt" ReadWriteMode)
            hClose
            (\h -> do
                  stdin' <- hDuplicate stdin
                  hDuplicateTo h stdin
                  finally action (hDuplicateTo stdin' stdin)            
            )
        )
        (removeFile "stdin.txt")

catchPrint = flip catch p where
    p :: SomeException -> IO ()
    p = print

+ Se usarán las siguientes librerías

In [3]:
import Test.QuickCheck
import System.IO
import Data.List (nub)
import System.Random (randomRIO)

# Programas interactivos

+ Los programas por lote no interactúan con los usuarios durante su ejecución.

+ Los programas interactivos durante su ejecución pueden leer datos del teclado
  y escribir resultados en la pantalla.

+ Problema:
    + Los programas interactivos tienen efectos laterales.
    + Los programa Haskell no tiene efectos laterales.

**Ejemplo de programa interactivo**

+ Especificación: El programa pide una cadena y dice el número de caracteres
  que tiene.

+ Ejemplo de sesión:

```sesion
ghci> longitudCadena
Escribe una cadena: "Hoy es lunes"
La cadena tiene 14 caracteres
```

+ Programa:

In [4]:
longitudCadena :: IO ()
longitudCadena = do
  putStr "Escribe una cadena: "
  xs <- getLine
  putStr "La cadena tiene "
  putStr (show (length xs))
  putStrLn " caracteres"

In [5]:
withStdin "Hoy es lunes" longitudCadena

Escribe una cadena: La cadena tiene 12 caracteres

# El tipo de las acciones de entrada/salida

+ En Haskell se pueden escribir programas interactivos usando tipos que
  distingan las expresiones puras de las *acciones* impuras que tienen
  efectos laterales.

+ `IO a` es el tipo de las acciones que devuelven un valor del tipo `a`.

+ Ejemplos:
    + `IO Char` es el tipo de las acciones que devuelven un carácter.
    + `IO ()` es el tipo de las acciones que no devuelven ningún valor.

# Acciones básicas

+ `getChar :: IO Char`   
  La acción `getChar` lee un carácter del teclado, lo muestra en
  la pantalla y lo devuelve como valor.

+ `putChar :: c -> IO ()`   
  La acción `putChar c` escribe el carácter `c` en la pantalla y
  no devuelve ningún valor.

+ `return a -> IO a`  
  La acción `return c` devuelve el valor `c` sin ninguna
  interacción.

+ Ejemplo:

```sesion
ghci> putChar 'b'
bghci> it
()
```

In [6]:
putChar 'b'

b

# Secuenciación

+ Una sucesión de acciones puede combinarse en una acción compuesta mediante
  expresiones `do`.

+ Ejemplo: El procedimiento `ejSecuenciacion` lee dos caracteres y devuelve el
  par formado por ellos. Por ejemplo,

```sesion
ghci> ejSecuenciacion
b f
('b','f')
```

In [7]:
ejSecuenciacion :: IO (Char,Char)
ejSecuenciacion = do
   x <- getChar
   getChar
   y <- getChar
   return (x,y)

In [8]:
withStdin "b f" ejSecuenciacion

('b','f')

# Primitivas derivadas

+ Lectura de cadenas del teclado:

```haskell
getLine :: IO String
getLine = do x <- getChar
             if x == '\n' then return []
                else do xs <- getLine
                        return (x:xs)
```

+ Escritura de cadenas en la pantalla:

```haskell
putStr :: String -> IO ()
putStr []     = return ()
putStr (x:xs) = do putChar x
                   putStr xs
```

+ Escritura de cadenas en la pantalla y salto de línea:

```haskell
putStrLn :: String -> IO ()
putStrLn xs = do putStr xs
                 putChar '\n'
```

+ Ejecución de una lista de acciones. Por ejemplo,

In [9]:
sequence_ [putStrLn "uno", putStrLn "dos"]

uno
dos

Su definición es

```haskell
sequence_ :: [IO a] -> IO ()
sequence_ []     = return ()
sequence_ (a:as) = do a
                      sequence_ as
```
 

**Ejemplo de programa con primitivas derivadas**

+ Especificación: El programa pide una cadena y dice el número de caracteres
  que tiene.

+ Ejemplo de sesión:

```sesion
ghci> longitudCadena
Escribe una cadena: "Hoy es lunes"
La cadena tiene 14 caracteres
```

+ Programa:

In [10]:
longitudCadena :: IO ()
longitudCadena = do
   putStr "Escribe una cadena: "
   xs <- getLine
   putStr "La cadena tiene "
   putStr (show (length xs))
   putStrLn " caracteres"

In [11]:
withStdin "Hoy es Lunes" longitudCadena

Escribe una cadena: La cadena tiene 12 caracteres

# Ejemplos de programas interactivos

## Juego de adivinación interactivo

+ Descripción: El programa le pide al jugador humano que piense un número entre
  1 y 100 y trata de adivinar el número que ha pensado planteándole conjeturas
  a las que el jugador humano responde con mayor, menor o exacto según que el
  número pensado sea mayor, menor o igual que el número conjeturado por la
  máquina.

+ Ejemplo de sesión:

```sesion
Main> juego
Piensa un numero entre el 1 y el 100.
Es 50? [mayor/menor/exacto] mayor
Es 75? [mayor/menor/exacto] menor
Es 62? [mayor/menor/exacto] mayor
Es 68? [mayor/menor/exacto] exacto
Fin del juego
```

+ Programa:

In [12]:
juego :: IO ()
juego = do
    putStrLn "Piensa un numero entre el 1 y el 100."
    adivina 1 100
    putStrLn "\nFin del juego"

adivina :: Int -> Int -> IO ()
adivina a b =
    do putStr ("\nEs " ++ show conjetura ++ "? [mayor/menor/exacto] ")
       s <- getLine
       case s of
         "mayor"  -> adivina (conjetura+1) b
         "menor"  -> adivina a (conjetura-1)
         "exacto" -> return ()
         _        -> adivina a b
    where
       conjetura = (a+b) `div` 2

In [13]:
withStdin (unlines ["mayor", "menor", "mayor", "exacto"]) (catchPrint juego)

Piensa un numero entre el 1 y el 100.

Es 50? [mayor/menor/exacto] 
Es 75? [mayor/menor/exacto] 
Es 62? [mayor/menor/exacto] 
Es 68? [mayor/menor/exacto] 
Fin del juego

+ Descripción: En el segundo juego la máquina genera un número aleatorio entre
  1 y 100 y le pide al jugador humano que adivine el número que ha pensado
  planteándole conjeturas a las que la máquina responde con mayor, menor o
  exacto según que el número pensado sea mayor, menor o igual que el número
  conjeturado por el jugador humano.

+ Ejemplo de sesión:

```sesion
Main> juego2
Tienes que adivinar un numero entre 1 y 100
Escribe un numero: 50
 es bajo.
Escribe un numero: 75
 es alto.
Escribe un numero: 62
 Exactamente
```

+ Se usa la librería de generación de números aleatorios:

In [14]:
import System.Random (randomRIO)

+ Programa:

In [15]:
juego2 :: IO ()
juego2 = do n <- randomRIO (1::Int,100)
            putStrLn "Tienes que adivinar un numero entre 1 y 100"
            adivina' n

adivina' :: Int -> IO ()
adivina' n = 
    do putStr "Escribe un numero: "
       c <- getLine
       let x = read c 
       case (compare x n) of
         LT -> do putStrLn " es bajo."
                  adivina' n
         GT -> do putStrLn " es alto."
                  adivina' n
         EQ -> putStrLn " Exactamente"

In [16]:
withStdin (unlines ["50", "75", "62"]) (catchPrint juego2)

Tienes que adivinar un numero entre 1 y 100
Escribe un numero:  es alto.
Escribe un numero:  es alto.
Escribe un numero:  es alto.
Escribe un numero: <stdin>: hGetLine: end of file

# Manejo de ficheros

**Lectura de ficheros con readFile**

+ Supongamos que el fichero `Ejemplo_1.txt` tiene el siguiente contenido

~~~
Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.
~~~

+ El procedimiento de lectura de ficheros:

In [17]:
:type readFile

readFile :: FilePath -> IO String

In [18]:
readFile "ejemplos/Ejemplo_1.txt"

"Este fichero tiene tres lineas\nesta es la segunda y\nesta es la tercera.\n"

In [19]:
putStrLn it

Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.

In [20]:
cs <- readFile "ejemplos/Ejemplo_1.txt"

In [21]:
putStrLn cs

Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.

+ El procedimiento `(muestraContenidoFichero f)` muestra en pantalla el
  contenido del fichero `f`. Por ejemplo,

```sesion
λ> muestraContenidoFichero "Ejemplo_1.txt"
Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.
```

El programa es

In [22]:
muestraContenidoFichero :: FilePath -> IO ()
muestraContenidoFichero f = do
  cs <- readFile f
  putStrLn cs

In [23]:
muestraContenidoFichero "ejemplos/Ejemplo_1.txt"

Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.

**Escritura en ficheros con writeFile**

+  El procedimiento de escritura de ficheros:

In [24]:
:type writeFile

writeFile :: FilePath -> String -> IO ()

In [25]:
texto = "Hay\ntres lineas\nde texto"

In [26]:
writeFile "ejemplos/Ejemplo_2.txt" texto

In [27]:
muestraContenidoFichero "ejemplos/Ejemplo_2.txt"

Hay
tres lineas
de texto

+ El procedimiento `(aMayucula f1 f2)` lee el contenido del fichero f1 y
  escribe su contenido en mayúscula en el fichero f2. Por ejemplo,

```sesion
λ> muestraContenidoFichero "Ejemplo_1.txt"
Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.

λ> aMayuscula "Ejemplo_1.txt" "Ejemplo_3.txt"
λ> muestraContenidoFichero "Ejemplo_3.txt"
ESTE FICHERO TIENE TRES LINEAS
ESTA ES LA SEGUNDA Y
ESTA ES LA TERCERA.
```

El programa es

In [28]:
import Data.Char (toUpper)
 
aMayuscula f1 f2 = do
  contenido <- readFile f1
  writeFile f2 (map toUpper contenido)  

In [29]:
muestraContenidoFichero "ejemplos/Ejemplo_1.txt"

Este fichero tiene tres lineas
esta es la segunda y
esta es la tercera.

In [30]:
aMayuscula "ejemplos/Ejemplo_1.txt" "ejemplos/Ejemplo_3.txt"

In [31]:
muestraContenidoFichero "ejemplos/Ejemplo_3.txt"

ESTE FICHERO TIENE TRES LINEAS
ESTA ES LA SEGUNDA Y
ESTA ES LA TERCERA.

+ El procedimiento `(ordenaFichero f1 f2)` lee el contenido del fichero
  f1 y escribe su contenido ordenado en el fichero f2. Por ejemplo, 

```sesion
λ> muestraContenidoFichero "Ejemplo_4a.txt"
Juan Ramos
Ana Ruiz
Luis Garcia
Blanca Perez

λ> ordenaFichero "Ejemplo_4a.txt" "Ejemplo_4b.txt"
λ> muestraContenidoFichero "Ejemplo_4b.txt"
Ana Ruiz
Blanca Perez
Juan Ramos
Luis Garcia
```

El programa es

In [32]:
import Data.List (sort)
 
ordenaFichero :: FilePath -> FilePath -> IO ()
ordenaFichero f1 f2 = do
  cs <- readFile f1
  writeFile f2 ((unlines . sort . lines) cs)

In [33]:
muestraContenidoFichero "ejemplos/Ejemplo_4a.txt"

Juan Ramos
Ana Ruiz
Luis Garcia
Blanca Perez

In [34]:
ordenaFichero "ejemplos/Ejemplo_4a.txt" "ejemplos/Ejemplo_4b.txt"

In [35]:
muestraContenidoFichero "ejemplos/Ejemplo_4b.txt"

Ana Ruiz
Blanca Perez
Juan Ramos
Luis Garcia

+ Las funciones lines y unlines

In [36]:
:type lines

lines :: String -> [String]

In [37]:
:type unlines

unlines :: [String] -> String

In [38]:
unlines ["ayer fue martes", "hoy es miercoles","de enero"] 

"ayer fue martes\nhoy es miercoles\nde enero\n"

In [39]:
lines it

["ayer fue martes","hoy es miercoles","de enero"]

+ Las funciones words y unwords

In [40]:
:type words

words :: String -> [String]

In [41]:
:type unwords

unwords :: [String] -> String

In [42]:
words "ayer fue   martes"

["ayer","fue","martes"]

In [43]:
unwords it

"ayer fue martes"

+ El procedimiento `(tablaCuadrados f n)` escribe en el fichero f los
  cuadrados de los n primeros números. Por ejemplo.

```sesion
λ> tablaCuadrados "cuadrados.txt" 9
λ> muestraContenidoFichero "cuadrados.txt"
(1,1) (2,4) (3,9) (4,16) (5,25) (6,36) (7,49) (8,64) (9,81)
```

El programa es

In [44]:
tablaCuadrados :: FilePath -> Int -> IO ()
tablaCuadrados f n =
  writeFile f (listaDeCuadrados n)
 
listaDeCuadrados :: Int -> String
listaDeCuadrados n =
  unwords (map show [(x,x*x) | x <- [1..n]])

In [45]:
tablaCuadrados "ejemplos/cuadrados.txt" 9
muestraContenidoFichero "ejemplos/cuadrados.txt"

(1,1) (2,4) (3,9) (4,16) (5,25) (6,36) (7,49) (8,64) (9,81)

+ El procedimiento (tablaCuadrados2 f n) escribe en el fichero f los
  cuadrados de los n  primeros números, uno por línea. Por ejemplo. 

```sesion
λ> tablaCuadrados2 "cuadrados.txt" 5
λ> muestraContenidoFichero "cuadrados.txt"
(1,1)
(2,4)
(3,9)
(4,16)
(5,25)
```

El programa es

In [46]:
tablaCuadrados2 :: FilePath -> Int -> IO ()
tablaCuadrados2 f n =
  writeFile f (listaDeCuadrados2 n)
 
listaDeCuadrados2 :: Int -> String
listaDeCuadrados2 n =
  unlines (map show [(x,x*x) | x <- [1..n]])

In [47]:
tablaCuadrados2 "ejemplos/cuadrados.txt" 5
muestraContenidoFichero "ejemplos/cuadrados.txt"

(1,1)
(2,4)
(3,9)
(4,16)
(5,25)

+ El procedimiento (tablaLogaritmos f ns) escribe en el fichero f los
  cuadrados de los números de ns, uno por línea. Por ejemplo. 

```sesion
λ> tablaLogaritmos "z.txt" [1,3..20]
λ> muestraContenidoFichero "z.txt"
+----+----------------+
| n  | log(n)         |
+----+----------------+
|  1 | 0.000000000000 |
|  3 | 1.098612288668 |
|  5 | 1.609437912434 |
|  7 | 1.945910149055 |
|  9 | 2.197224577336 |
| 11 | 2.397895272798 |
| 13 | 2.564949357462 |
| 15 | 2.708050201102 |
| 17 | 2.833213344056 |
| 19 | 2.944438979166 |
+----+----------------+
```

El programa es

In [48]:
import Text.Printf
 
tablaLogaritmos :: FilePath -> [Int] -> IO ()
tablaLogaritmos f ns = do
  writeFile f (tablaLogaritmosAux ns)
 
tablaLogaritmosAux :: [Int] -> String
tablaLogaritmosAux ns =
     linea
  ++ cabecera
  ++ linea
  ++ concat [printf "| %2d | %.12f |\n" n x
            | n <- ns
            , let x = log (fromIntegral n) :: Double]
  ++ linea
 
linea, cabecera :: String
linea    = "+----+----------------+\n"
cabecera = "| n  | log(n)         |\n"

In [49]:
tablaLogaritmos "ejemplos/z.txt" [1,3..20]
muestraContenidoFichero "ejemplos/z.txt"

+----+----------------+
| n  | log(n)         |
+----+----------------+
|  1 | 0.000000000000 |
|  3 | 1.098612288668 |
|  5 | 1.609437912434 |
|  7 | 1.945910149055 |
|  9 | 2.197224577336 |
| 11 | 2.397895272798 |
| 13 | 2.564949357462 |
| 15 | 2.708050201102 |
| 17 | 2.833213344056 |
| 19 | 2.944438979166 |
+----+----------------+

# Bibliografía

+ J.A. Alonso.
  [Gráficas con GNUplot en IHaskell
  ](https://github.com/jaalonso/GraficasEnIHaskell)

+ H. Daumé III.
  [Yet Another Haskell Tutorial
  ](http://www.cs.utah.edu/~hal/docs/daume02yaht.pdf), 2006. 
    + Cap. 5: Basic Input/Output.

+ G. Hutton.
  *Programming in Haskell*. 
  Cambridge University Press, 2007.
    + Cap. 9: Interactive programs.

+ M. Lipovača 
  *¡Aprende Haskell por el bien de todos!*
    + Cap. 9.2: [Ficheros y flujos de datos](http://bit.ly/1O3XDBW)

+ B. O’Sullivan, J. Goerzen y D. Stewart. 
  *Real World Haskell*. 
  O'Reilly, 2009.
    + Cap. 7: [I/O](http://bit.ly/1O3XIWi).
  
+ B.C. Ruiz, F. Gutiérrez, P. Guerrero y J.E. Gallardo.
  *Razonando con Haskell*.
  Thompson, 2004.
    + Cap. 7: Entrada y salida.

+ S. Thompson.
  *Haskell: The Craft of Functional Programming*, Second Edition.
  Addison-Wesley, 1999.
    + Cap. 18: Programming with actions.
